## Importing Python Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler

##  Loading eBay stock price dataset
##  and Preparation of training set

In [2]:
dataset_train = pd.read_csv('ebay_stockprice_train.csv')
training_set = dataset_train.iloc[:, 1:2].values

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

X_train = []
y_train = []

for i in range(50, 1000):
    X_train.append(training_set_scaled[i-50:i, 0])
    y_train.append(training_set_scaled[i, 0])

## Getting the training data in shape

In [3]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Creating the model for ebay stock price prediction

In [4]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.5))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.5))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

print(regressor.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 50)            10400     
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 50, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 50, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 50, 50)            0         
                                                                 
 lstm_3 (LSTM)               (None, 50)                2

## Training the Model

In [5]:
regressor.fit(X_train, y_train, epochs = 150, batch_size = 30)

Epoch 1/150
32/32 [==============================] - 8s 56ms/step - loss: 0.0348
Epoch 2/150
32/32 [==============================] - 2s 52ms/step - loss: 0.0128
Epoch 3/150
32/32 [==============================] - 2s 53ms/step - loss: 0.0120
Epoch 4/150
32/32 [==============================] - 2s 51ms/step - loss: 0.0102
Epoch 5/150
32/32 [==============================] - 2s 60ms/step - loss: 0.0099
Epoch 6/150
32/32 [==============================] - 2s 71ms/step - loss: 0.0092
Epoch 7/150
32/32 [==============================] - 2s 73ms/step - loss: 0.0090
Epoch 8/150
32/32 [==============================] - 2s 71ms/step - loss: 0.0077
Epoch 9/150
32/32 [==============================] - 2s 72ms/step - loss: 0.0081
Epoch 10/150
32/32 [==============================] - 2s 64ms/step - loss: 0.0096
Epoch 11/150
32/32 [==============================] - 2s 55ms/step - loss: 0.0084
Epoch 12/150
32/32 [==============================] - 2s 55ms/step - loss: 0.0077
Epoch 13/150
32/32 [=====

32/32 [==============================] - 2s 71ms/step - loss: 0.0030
Epoch 102/150
32/32 [==============================] - 2s 68ms/step - loss: 0.0031
Epoch 103/150
32/32 [==============================] - 2s 63ms/step - loss: 0.0030
Epoch 104/150
32/32 [==============================] - 2s 70ms/step - loss: 0.0028
Epoch 105/150
32/32 [==============================] - 2s 70ms/step - loss: 0.0027
Epoch 106/150
32/32 [==============================] - 2s 72ms/step - loss: 0.0028
Epoch 107/150
32/32 [==============================] - 2s 73ms/step - loss: 0.0029
Epoch 108/150
32/32 [==============================] - 2s 70ms/step - loss: 0.0026
Epoch 109/150
32/32 [==============================] - 2s 56ms/step - loss: 0.0026
Epoch 110/150
32/32 [==============================] - 2s 54ms/step - loss: 0.0026
Epoch 111/150
32/32 [==============================] - 2s 54ms/step - loss: 0.0029
Epoch 112/150
32/32 [==============================] - 2s 57ms/step - loss: 0.0028
Epoch 113/150
32/3

## Loading the test dataset

In [6]:
ebay_test_dataset = pd.read_csv('ebay_stockprice_test.csv')
original_stock_price = ebay_test_dataset.iloc[:, 1:2].values

## Genereating the predicted stock price on test dataset

In [7]:
dataset_total = pd.concat((dataset_train['Open'], ebay_test_dataset['Open']), axis = 0)
input_data = dataset_total[len(dataset_total) - len(ebay_test_dataset) - 50:].values
input_data = input_data.reshape(-1,1)
input_data = sc.transform(input_data)
X_test = []
for i in range(50, 80):
    X_test.append(input_data[i-50:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

<ipython-input-7-5dc153b5793f>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(X_test)


IndexError: tuple index out of range

## Generating the plot for predicted data and original data

In [ ]:
plt.plot(original_stock_price, color = 'red', label = 'Original eBay Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted eBay Stock Price')
plt.title('eBay Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('eBay Stock Price')
plt.legend()
plt.show()